In [1]:
from neuralop.models import FNO
import torch
import pathlib

from acoustic_no.data import AcousticDataset
from acoustic_no.cno.cno_model import CNOModel
from acoustic_no.utils.eval import evaluate_models


In [2]:
# Use the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the dataset
dataset = AcousticDataset(
    data_dir=pathlib.Path("../resources/small_dataset/testing"),
    depth=64
)
print(f"Dataset size: {len(dataset)}")

depth = dataset.depth
print(f"Dataset depth: {depth}")


Using device: cuda
Dataset size: 8757
Dataset depth: 64


In [3]:
fno = FNO(
    n_modes=(16, 16),
    in_channels=depth * 3 + 1,
    out_channels=depth,
    n_layers=8,
    hidden_channels=128,
    projection_channel_ratio=2,
)
fno.load_state_dict(
    torch.load(
        "../resources/models/fno_baseline/best_model_state_dict.pt",
        map_location=device,
        weights_only=False
    )
)
fno.to(device)

tfno = FNO(
    n_modes=(16, 16),
    in_channels=depth * 3 + 1,
    out_channels=depth,
    n_layers=8,
    hidden_channels=128,
    projection_channel_ratio=2,
    factorization="Tucker"
)
tfno.load_state_dict(
    torch.load(
        "../resources/models/tfno/best_model_state_dict.pt",
        map_location=device,
        weights_only=False
    )
)
tfno.to(device)

cno = CNOModel(
    input_channels=depth * 3 + 1,
    hidden_channels=[128, 128, 256],
    layer_sizes=[3, 3, 4],
    output_channels=depth
)
cno.load_state_dict(
    torch.load(
        "../resources/models/cno/best_model.pth",
        map_location=device
    )
)
cno.to(device)

pass


In [4]:
models = {
    "FNO": fno,
    "TFNO": tfno,
    "CNO": cno
}

results = evaluate_models(models, dataset, device)


Evaluating: 100%|██████████| 8757/8757 [04:30<00:00, 32.35sample/s]

----------------------------------------
Average results for model 'FNO':
  MSE:     13.932867
  L2 Loss: 49.230886
  H1 Loss: 51.069296
----------------------------------------
Average results for model 'TFNO':
  MSE:     18.452117
  L2 Loss: 52.055132
  H1 Loss: 54.566852
----------------------------------------
Average results for model 'CNO':
  MSE:     1.980019
  L2 Loss: 20.405886
  H1 Loss: 21.855785
